In [21]:
import matplotlib.pyplot as plt
import numpy as np
from mne import set_log_level
from mne.io import read_raw_eeglab

from pycrostates.cluster import ModKMeans
from pycrostates.datasets import lemon


set_log_level("ERROR")  # reduce verbosity

raw_fname = lemon.data_path(subject_id="010017", condition="EO")
raw = read_raw_eeglab(raw_fname, preload=True)
raw.crop(0, 180)
raw.pick("eeg")
raw.set_eeg_reference("average")

<RawEEGLAB | sub-010017_EO.fdt, 61 x 45001 (180.0 s), ~21.0 MB, data loaded>

In [22]:
from pycrostates.preprocessing import extract_gfp_peaks


cluster_numbers = [2,3,4,5,6,8,10,12,15,20]

gfp_peaks = extract_gfp_peaks(raw)

gevs = list()
image_files=list()
for k, n_clusters in enumerate(cluster_numbers):
    # fit K-means algorithm with a set number of cluster centers
    ModK = ModKMeans(n_clusters=n_clusters, random_state=42)
    ModK.fit(gfp_peaks, n_jobs=6, verbose="WARNING")
    gevs.append(ModK.GEV_)
    fig = ModK.plot(show=False)
    image = f'cluster_centers_{k}.png'
    fig.savefig(image)   # save the figure to file
    plt.close(fig) 
    image_files.append(image)

KeyboardInterrupt: 

In [ ]:
import numpy as np
from bokeh.io import output_file, show
from bokeh.models import FuncTickFormatter
from bokeh.plotting import figure
from bokeh.transform import linear_cmap
from bokeh.palettes import Viridis256
from PIL import Image
import base64
from io import BytesIO
from bokeh.plotting import figure, output_file, show, ColumnDataSource
from bokeh.models import HoverTool


# Prepare the data
x = list(cluster_numbers)
y = gevs

# Load the images
images = ["./" + file for file in image_files]

hover = HoverTool(
        tooltips="""
        <div>
            <div>
                <img
                    src="@imgs" alt="@imgs"
                    style="float: left; margin: 0px 15px 15px 0px;"
                    border="2"
                ></img>
            </div>
        </div>
        """
    )


# Create a Bokeh ColumnDataSource
source = ColumnDataSource(data=dict(x=x, y=y, imgs=images))

# Create a scatter plot
p = figure(title="Scatter Plot with Overlaid Images", x_axis_label="X", y_axis_label="Y", tools=[hover])
p.scatter(x='x', y='y', size=10, color='red', source=source)

# Display the plot
show(p)

In [19]:
def RC(x, A, B, t):
    y = A * (1 - np.exp(-x/t)) + B
    return y

In [20]:
from scipy.optimize import curve_fit

parameters, covariance = curve_fit(RC, list(cluster_numbers), gevs)